In [2]:
import os
import numpy as np
import librosa
import matplotlib.pyplot as plt
import librosa.display
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization




In [3]:
emotion_dict ={
  '01':'Neutral',
  '02':'Calm',
  '03':'happy',
  '04':'Sad',
  '05':'angry',
  '06':'Fear',
  '07':'Disgust',
  '08':'suprise',
}

In [4]:
def extract_mfcc(file_path):
    audio, sr = librosa.load(file_path, duration=3, offset=0.5)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    return np.mean(mfcc.T, axis=0)


In [23]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
X = []
y = []

data_path = "/content/audio_speech_actors_01-24"




for actor in os.listdir(data_path):
    actor_path = os.path.join(data_path, actor)

    for file in os.listdir(actor_path):
        file_path = os.path.join(actor_path, file)

        emotion = file.split("-")[2]

        X.append(extract_mfcc(file_path))
        y.append(emotion)

X = np.array(X)
y = np.array(y)


In [27]:
import os

data_path = "/content/audio_speech_actors_01-24"

for actor in os.listdir(data_path):
    actor_path = os.path.join(data_path, actor)
    print(actor_path)


/content/audio_speech_actors_01-24/Actor_24
/content/audio_speech_actors_01-24/Actor_12
/content/audio_speech_actors_01-24/Actor_11
/content/audio_speech_actors_01-24/Actor_20
/content/audio_speech_actors_01-24/Actor_19
/content/audio_speech_actors_01-24/Actor_09
/content/audio_speech_actors_01-24/Actor_18
/content/audio_speech_actors_01-24/Actor_21
/content/audio_speech_actors_01-24/Actor_10
/content/audio_speech_actors_01-24/Actor_02
/content/audio_speech_actors_01-24/Actor_22
/content/audio_speech_actors_01-24/Actor_13
/content/audio_speech_actors_01-24/Actor_15
/content/audio_speech_actors_01-24/Actor_01
/content/audio_speech_actors_01-24/Actor_17
/content/audio_speech_actors_01-24/Actor_05
/content/audio_speech_actors_01-24/Actor_23
/content/audio_speech_actors_01-24/Actor_08
/content/audio_speech_actors_01-24/Actor_07
/content/audio_speech_actors_01-24/Actor_06
/content/audio_speech_actors_01-24/Actor_04
/content/audio_speech_actors_01-24/Actor_16
/content/audio_speech_actors_01-

In [28]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)


In [29]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)


In [30]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42
)


In [31]:
model = Sequential()

model.add(Dense(512, input_shape=(40,), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(128, activation='relu'))

model.add(Dense(8, activation='softmax'))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
print(X_train.shape)
print(y_train.shape)


(1152, 40)
(1152, 8)


In [34]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [35]:
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_data=(X_test, y_test)
)


Epoch 1/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.2004 - loss: 2.2873 - val_accuracy: 0.3160 - val_loss: 1.9033
Epoch 2/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3690 - loss: 1.6726 - val_accuracy: 0.3750 - val_loss: 1.7796
Epoch 3/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5100 - loss: 1.3590 - val_accuracy: 0.3889 - val_loss: 1.6649
Epoch 4/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5575 - loss: 1.2336 - val_accuracy: 0.5139 - val_loss: 1.5196
Epoch 5/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5800 - loss: 1.1314 - val_accuracy: 0.5104 - val_loss: 1.3918
Epoch 6/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5961 - loss: 1.0240 - val_accuracy: 0.5799 - val_loss: 1.2846
Epoch 7/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6310 - loss: 0.9804 - val_accuracy: 0.5903 - val_loss: 1.1885
Epoch 8/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6674 - loss: 0.9572 - val_accuracy: 0.6076 - 

In [36]:
loss , acc =model.evaluate(X_test,y_test)
print("Loss: ",loss)
print("Accuracy: ",acc)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7419 - loss: 1.1885
Loss:  1.2846522331237793
Accuracy:  0.7465277910232544


In [37]:
loss , acc =model.evaluate(X_test,y_test)
print("Loss: ",loss)
print("Accuracy: ",acc)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7419 - loss: 1.1885 
Loss:  1.2846522331237793
Accuracy:  0.7465277910232544


In [38]:
model.save("emotion_model.h5")


In [ ]:
file_path = "/Suprise.m4a"

sample = extract_mfcc(file_path)
sample = np.expand_dims(sample, axis=0)

prediction = model.predict(sample)
emotion = encoder.inverse_transform([np.argmax(prediction)])

print("Predicted Emotion:", emotion[0])


In [ ]:
file_path = "/content/happy2.wav.m4a"

sample = extract_mfcc(file_path)
sample = np.expand_dims(sample, axis=0)

prediction = model.predict(sample)
emotion = encoder.inverse_transform([np.argmax(prediction)])

print("Predicted Emotion:", emotion[0])